In [1]:
from pathlib import Path

data_dir = Path.cwd() / "data" / "uncompressed"

mimiic_dir = data_dir / "mimiciv" / "3.1"

In [2]:
import pandas as pd
import dask

pd.options.mode.string_storage = "pyarrow"
dask.config.set({"dataframe.convert-string": True})

In [3]:
import dask.dataframe as dd

from dask.distributed import Client

client = Client(n_workers=6)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 18,Total memory: 47.05 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45941,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:38385,Total threads: 3
Dashboard: http://127.0.0.1:44559/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:42921,


2025-09-19 08:53:31,609 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 2b6eddb01dd3c96f07a914d97aad6f2e initialized by task ('shuffle-transfer-2b6eddb01dd3c96f07a914d97aad6f2e', 1) executed on worker tcp://127.0.0.1:44709
2025-09-19 08:53:31,809 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 2b6eddb01dd3c96f07a914d97aad6f2e deactivated due to stimulus 'task-finished-1758272011.8072712'


In [5]:
from open_icu.config.utils import load_yaml_configs
from open_icu.config.source import SourceConfig

configs = load_yaml_configs(Path.cwd().parent / "configs" / "source", SourceConfig)
configs

[SourceConfig(name='mimic', version='3.1', tables=[TableConfig(name='medications', path='icu/inputevents.csv', fields=[FieldConfig(field='subject_id', type='int64', constant=None), FieldConfig(field='hadm_id', type='int64', constant=None), FieldConfig(field='stay_id', type='int64', constant=None), FieldConfig(field='endtime', type='datetime', constant=None), FieldConfig(field='amount', type='float32', constant=None), FieldConfig(field='amountuom', type='string', constant=None), FieldConfig(field='starttime', type='datetime', constant=None), FieldConfig(field='rateuom', type='string', constant=None), FieldConfig(field='rate', type='float32', constant=None), FieldConfig(field='itemid', type='int64', constant=None)], join=[JoinConfig(name='d_items', path='icu/d_items.csv', fields=[FieldConfig(field='itemid', type='int64', constant=None), FieldConfig(field='label', type='string', constant=None)], both_on=['itemid'], left_on=[], right_on=[], how='left', join_params={'on': ['itemid']})], eve

In [12]:
from open_icu.meds.processor import process_table


process_table(configs[0].tables[0], mimiic_dir, Path.cwd() / "output", "mimic")

In [ ]:
from open_icu.meds.processor import process_table


for config in configs[:1]:
    for table in config.tables:
        if table.name == "chartevents":
            continue

        process_table(table, mimiic_dir, Path.cwd() / "output", "mimic")

In [14]:
import dask.dataframe as dd
from pathlib import Path

df = dd.read_parquet(Path.cwd() / "output" / "data" / "mimic_medications_dosage_0.parquet").head()
df

,subject_id,time,code,numeric_value,text_value,hadm_id,stay_id
0,10000032,2180-07-23 17:01:00,PO Intake//mL,200.0,,29079034,39553978
1,10000032,2180-07-23 17:30:00,Albumin 25%//mL,50.0,,29079034,39553978
2,10000032,2180-07-23 18:03:00,Albumin 25%//mL,50.0,,29079034,39553978
3,10000032,2180-07-23 18:57:00,PO Intake//mL,100.0,,29079034,39553978
4,10000032,2180-07-23 21:11:00,PO Intake//mL,100.0,,29079034,39553978


In [15]:
dd.read_parquet(Path.cwd() / "output" / "data" / "mimic_medications_rate_0.parquet").head()

,subject_id,time,code,numeric_value,text_value,hadm_id,stay_id
0,10000032,2180-07-23 17:00:00,Albumin 25%//mL/hour,100.000000,,29079034,39553978
1,10000032,2180-07-23 17:33:00,Albumin 25%//mL/hour,100.000000,,29079034,39553978
2,10000690,2150-11-02 20:00:00,Phenylephrine//mcg/kg/min,0.600106,,25860671,37081114
3,10000690,2150-11-02 20:00:00,NaCl 0.9%//mL/hour,8.296460,,25860671,37081114
4,10000690,2150-11-02 22:45:00,Phenylephrine//mcg/kg/min,0.499987,,25860671,37081114


In [ ]:
dd.read_parquet(Path.cwd() / "output" / "data" / "mimic_chartevents_chartevent_0.parquet").head()

In [ ]:
df = dd.read_parquet(Path.cwd() / "output" / "metadata" / "codes.parquet").compute()
df

In [ ]:
dd.read_parquet(Path.cwd() / "output" / "data" / "mimic_icustays_icu_admission_0.parquet").head()

In [ ]:
dd.read_parquet(Path.cwd() / "output" / "data" / "mimic_icustays_icu_discharge_0.parquet").head()

In [ ]:
dd.read_parquet(Path.cwd() / "output" / "data" / "mimic_icustays_icu_length_of_stay_0.parquet").head()